In [1]:
!pip install scikit-learn


In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import classification_report


In [3]:
image_dir = Path(r'/Users/dipak/Downloads/archive (10)/images')  #Points to your food images folder
filepaths = list(image_dir.glob(r'**/*.jpg'))  # finds all .jpg files recursively in all subfolders Result: A list of all image paths
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
'''For each image path, it extracts the parent folder name as the label (food type)
Example: If image is at food/images/pizza/image1.jpg, the label is "pizza"
This assumes each food type is in its own subfolder'''
filepaths = pd.Series(filepaths, name='Filepath').astype(str) #Combines filepaths and labels into a pandas DataFrame with two columns: Filepath and Label
labels = pd.Series(labels, name='Label')
images = pd.concat([filepaths, labels], axis=1)
category_samples = [] #For each unique food category, it randomly selects 100 images
for category in images['Label'].unique(): 
    category_slice = images.query("Label == @category")
    category_samples.append(category_slice.sample(100, random_state=1))
image_df = pd.concat(category_samples, axis=0).sample(
    frac=1.0, random_state=1).reset_index(drop=True)

In [4]:
image_df['Label'].value_counts()

Label
pork_chop        100
bread_pudding    100
club_sandwich    100
french_fries     100
beef_tartare     100
                ... 
creme_brulee     100
hummus           100
clam_chowder     100
caprese_salad    100
foie_gras        100
Name: count, Length: 101, dtype: int64

In [5]:
train_df, test_df = train_test_split(
image_df, train_size=0.7, shuffle=True, random_state=42)
#Splits your balanced dataset into training (70%) and testing (30%) sets
#shuffle=True randomly mixes before splitting (prevents sequential bias)

In [6]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator( #Loads and processes images in batches during training
preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input, #Applies MobileNetV2's specific preprocessing (normalizes pixel values to match what the model expects)
validation_split=0.2
)#From the training data, 20% is reserved for validation
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [7]:
'''This code is the bridge between your data and the model. It loads actual image files from disk, preprocesses them, and organizes them into batches ready for training. Let me explain each part:

What flow_from_dataframe() Does
It reads image files from your DataFrame and converts them into batches that TensorFlow can use. Think of it as a smart data loader.'''

train_images = train_generator.flow_from_dataframe(
dataframe=train_df,
x_col='Filepath',
y_col='Label',
target_size=(224, 224),
color_mode='rgb',
class_mode='categorical',
batch_size=32,
shuffle=True,
seed=42,
subset='training'
)
'''Loads images from train_df paths

Resizes each to 224×224 (MobileNetV2's expected input size)

Shuffles order to prevent overfitting to image sequence

Creates batches of 32 images at a time

Subset='training' means: Use only the 80% portion (excluding validation)
'''
val_images = train_generator.flow_from_dataframe(
dataframe=train_df,
x_col='Filepath',
y_col='Label',
target_size=(224, 224),
color_mode='rgb',
class_mode='categorical',
batch_size=32,
shuffle=True,
seed=42,
subset='validation'
)
test_images = test_generator.flow_from_dataframe(
dataframe=test_df,
x_col='Filepath',
y_col='Label',
target_size=(224, 224),
color_mode='rgb',
class_mode='categorical',
batch_size=32,
shuffle=False
)

Found 5656 validated image filenames belonging to 101 classes.
Found 1414 validated image filenames belonging to 101 classes.
Found 3030 validated image filenames belonging to 101 classes.


In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(  #loading a pretrained model that already knows how to recognize patterns in images. Let me break it down:
input_shape=(224, 224, 3),
include_top=False,
weights='imagenet',
pooling='avg'
)
pretrained_model.trainable = False

In [10]:
inputs = pretrained_model.input
x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(101, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)
print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,451,493 (9.35 MB)

 Trainable params: 193,509 (755.89 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

None


In [11]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True)])

Epoch 1/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 48s 256ms/step - accuracy: 0.1363 - loss: 3.8841 - val_accuracy: 0.2786 - val_loss: 2.9732
Epoch 2/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 45s 257ms/step - accuracy: 0.3780 - loss: 2.4391 - val_accuracy: 0.3656 - val_loss: 2.4784
Epoch 3/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 44s 251ms/step - accuracy: 0.5095 - loss: 1.8704 - val_accuracy: 0.4123 - val_loss: 2.3644
Epoch 4/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 46s 260ms/step - accuracy: 0.5964 - loss: 1.4992 - val_accuracy: 0.4399 - val_loss: 2.3084
Epoch 5/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 46s 258ms/step - accuracy: 0.6773 - loss: 1.1930 - val_accuracy: 0.4392 - val_loss: 2.3038
Epoch 6/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 46s 262ms/step - accuracy: 0.7550 - loss: 0.9328 - val_accuracy: 0.4364 - val_loss: 2.3854
Epoch 7/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 50s 282ms/step - accuracy: 0.8108 - loss: 0.7254 - val_accuracy: 0.4342 - val_loss: 2.4972
Epoch 8/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 52s 292ms/step - accuracy: 0.8679 - loss: 0

In [12]:
results = model.evaluate(test_images, verbose=0)
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

Test Accuracy: 42.41%
